In [ ]:
from math import sqrt, cos, sin, pi, sqrt, floor, ceil
import rtsvg
rt = rtsvg.RACETrack()

svg_hdr     = '<svg x="0" y="0" width="256" height="256" viewBox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white" />'
svg_ftr     = '</svg>'

#
# radicalCenter() - generated from Google Gemini
#
def radicalCenter(c0, c1, c2):
  """
  Finds the radical center of three circles.

  Args:
    c0: A tuple representing the first circle (x, y, r).
    c1: A tuple representing the second circle (x, y, r).
    c2: A tuple representing the third circle (x, y, r).

  Returns:
    A tuple representing the coordinates of the radical center (x, y), 
    or None if no radical center exists.
  """

  def power_of_point(circle, point):
    """Calculates the power of a point with respect to a circle."""
    x0, y0, r0 = circle
    x, y = point
    distance_squared = (x - x0)**2 + (y - y0)**2
    return distance_squared - r0**2

  def find_radical_axis(c1, c2):
    """Finds the equation of the radical axis of two circles."""
    x1, y1, r1 = c1
    x2, y2, r2 = c2
    # Equation of the radical axis: 
    # 2*(x1 - x2)*x + 2*(y1 - y2)*y + (r2**2 - r1**2 - x2**2 - y2**2 + x1**2 + y1**2) = 0
    A = 2 * (x1 - x2)
    B = 2 * (y1 - y2)
    C = r2**2 - r1**2 - x2**2 - y2**2 + x1**2 + y1**2
    return A, B, C

  # Find the equations of the radical axes
  axis12 = find_radical_axis(c0, c1)
  axis13 = find_radical_axis(c0, c2)
  # Solve the system of linear equations to find the intersection point
  A1, B1, C1 = axis12
  A2, B2, C2 = axis13
  # Check for parallel lines
  if A1 * B2 - A2 * B1 == 0: 
    return None  # No unique solution, circles are coaxial
  # Use Cramer's Rule to solve for x and y
  D = A1 * B2 - A2 * B1
  Dx = C1 * B2 - C2 * B1
  Dy = A1 * C2 - A2 * C1
  x = Dx / D
  y = Dy / D
  return x, y

def circleIntersections(c0, c1):
  """
  Finds the intersection points of two overlapping circles.

  Args:
    c0: A tuple representing the first circle (x0, y0, r0).
    c1: A tuple representing the second circle (x1, y1, r1).

  Returns:
    A list of tuples, each representing an intersection point (x, y). 
    Returns an empty list if no intersections exist.
  """
  (x0, y0, r0) = c0
  (x1, y1, r1) = c1
  # Calculate the distance between the centers
  d = sqrt((x1 - x0)**2 + (y1 - y0)**2)
  # Check for non-intersecting, tangent, or coincident circles
  if d > r0 + r1:         return []  # No intersection
  if d < abs(r0 - r1):    return []  # No intersection (one circle entirely within the other)
  if d == 0 and r0 == r1: return []  # Coincident circles
  # Calculate the distance from the center of c0 to the line segment connecting the centers
  a = (r0**2 - r1**2 + d**2) / (2 * d)
  # Calculate the coordinates of the point on the line segment connecting the centers
  x2 = x0 + a * (x1 - x0) / d
  y2 = y0 + a * (y1 - y0) / d
  # Calculate the height of the right triangle
  h = sqrt(r0**2 - a**2)
  # Calculate the coordinates of the intersection points
  x3 = x2 + h * (y1 - y0) / d
  y3 = y2 - h * (x1 - x0) / d
  x4 = x2 - h * (y1 - y0) / d
  y4 = y2 + h * (x1 - x0) / d
  return [(x3, y3), (x4, y4)]

def circleTangents(c, p):
  cx, cy = (c[0] + p[0])/2.0, (c[1] + p[1])/2.0
  r      = sqrt((c[0]-p[0])**2 + (c[1]-p[1])**2)/2.0
  points = circleIntersections(c, (cx, cy, r))
  return [(p,points[0]), (p,points[1])]

def orthogonalCircle(circles):
  xy_radical_center = radicalCenter(circles[0], circles[1], circles[2])
  _segments_        = circleTangents(circles[0], xy_radical_center)
  l                 = rt.segmentLength(_segments_[0])
  return (xy_radical_center[0], xy_radical_center[1], l)

def outerCircleTangents(c0, c1):
  if c0[2] > c1[2]: c0, c1 = c1, c0
  c_inner    = (c1[0], c1[1], c1[2]-c0[2])
  _segments_ = circleTangents(c_inner, c0)
  _finals_   = []
  for _segment_ in _segments_:
    _segment_[0] # this should be the center of c0
    _segment_[1] # this should be the delta out from the center of c1
    _uv_ = rt.unitVector(((c1[0], c1[1]), (_segment_[1][0], _segment_[1][1])))
    _finals_.append(((c0[0]+_uv_[0]*c0[2], c0[1]+_uv_[1]*c0[2]),(c1[0]+_uv_[0]*c1[2], c1[1]+_uv_[1]*c1[2])))
  return _finals_

def innerCircleTangents(c0, c1):
  if c0[2] > c1[2]: c0, c1 = c1, c0
  c_inner    = (c1[0], c1[1], c1[2]+c0[2]) # sign difference
  _segments_ = circleTangents(c_inner, c0)
  _finals_   = []
  for _segment_ in _segments_:
    _segment_[0] # this should be the center of c0
    _segment_[1] # this should be the delta out from the center of c1
    _uv_ = rt.unitVector(((c1[0], c1[1]), (_segment_[1][0], _segment_[1][1])))
    _finals_.append(((c0[0]-_uv_[0]*c0[2], c0[1]-_uv_[1]*c0[2]),(c1[0]+_uv_[0]*c1[2], c1[1]+_uv_[1]*c1[2]))) # also sign difference
  return _finals_

def svgX(xy, l=1.0, color='#000000', stroke_width=0.5):
  return f'<line x1="{xy[0]-l}" y1="{xy[1]-l}" x2="{xy[0]+l}" y2="{xy[1]+l}" stroke="{color}" stroke-width="{stroke_width}" />' + \
         f'<line x1="{xy[0]+l}" y1="{xy[1]-l}" x2="{xy[0]-l}" y2="{xy[1]+l}" stroke="{color}" stroke-width="{stroke_width}" />'


In [ ]:
#
# Version of the algorithm described here:
# https://www.cut-the-knot.org/Curriculum/Geometry/GeoGebra/CCC_Gergonne.shtml#solution
#
circles = [(45,45,8),(70,70,5),(40,70,3)]
svg_circles = []
for i in range(len(circles)): svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_orthogonal_circle = []
# Step 1: Find the orthogonal circle
circle_orthogonal = orthogonalCircle(circles)
# Step 2 & 3: Find the axis of similtude
_segments_ = outerCircleTangents(circles[0], circles[1])
R = rt.intersectionPoint(_segments_[0], _segments_[1])
_segments_ = outerCircleTangents(circles[0], circles[2])
Q = rt.intersectionPoint(_segments_[0], _segments_[1])
_segments_ = outerCircleTangents(circles[2], circles[1])
P = rt.intersectionPoint(_segments_[0], _segments_[1])
# Step 4: Draw the tangent lines
_segments_ = circleTangents(circles[2], R)
S1,T1      = _segments_[0][1], _segments_[1][1]
_segments_ = circleTangents(circles[1], Q)
S2,T2      = _segments_[0][1], _segments_[1][1]
_segments_ = circleTangents(circles[0], P)
S3,T3      = _segments_[0][1], _segments_[1][1]
# Step 5: draw the inner and outer circles
_inner_ = rt.threePointCircle(S1,S2,S3)
_outer_ = rt.threePointCircle(T1,T2,T3)
svg_orthogonal_circle.append(f'<circle cx="{_inner_[0]}" cy="{_inner_[1]}" r="{_inner_[2]}" fill="none" stroke="black" stroke-width="0.1" />')
svg_orthogonal_circle.append(f'<circle cx="{_outer_[0]}" cy="{_outer_[1]}" r="{_outer_[2]}" fill="none" stroke="black" stroke-width="0.1" />')
svg_orthogonal_circle.append(svgX(S1,color='green')+svgX(S2,color='green')+svgX(S3,color='green'))
svg_orthogonal_circle.append(svgX(T1)+svgX(T2)+svgX(T3))
rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_orthogonal_circle) + svg_ftr])

In [ ]:
import random
def makeInscribableCircleExample(_xyr_=(60.0, 60.0, 10.0), r_min=10.0, r_max=30.0):
    overlaps = True
    while overlaps:
        overlaps = False
        _circles_ = []
        for i in range(3):
            a, r = random.random()*2*pi, r_min + random.random() * (r_max - r_min)
            _circle_ = (_xyr_[0]+(_xyr_[2] + r)*cos(a), _xyr_[1]+(r + _xyr_[2])*sin(a), r)
            _circles_.append(_circle_)
        for i in range(3):
            for j in range(i+1, 3):
                if rt.segmentLength((_circles_[i][0:2], _circles_[j][0:2])) < _circles_[i][2] + _circles_[j][2]:
                    overlaps = True
    return _xyr_, _circles_
_inscribed_, _others_ = makeInscribableCircleExample()
svg_example = []
for i in range(len(_others_)): 
    svg_example.append(f'<circle cx="{_others_[i][0]}" cy="{_others_[i][1]}" r="{_others_[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.4" />')
svg_example.append(f'<circle cx="{_inscribed_[0]}" cy="{_inscribed_[1]}" r="{_inscribed_[2]}" fill="none" stroke="black" stroke-width="0.4" />')
rt.tile([svg_hdr + ''.join(svg_example) + svg_ftr])

In [ ]:
circles = _others_
svg_orthogonal_circle = []
# Step 1: Find the orthogonal circle
circle_orthogonal = orthogonalCircle(circles)
# Step 2 & 3: Find the axis of similtude
_segments_ = outerCircleTangents(circles[0], circles[1])
R = rt.intersectionPoint(_segments_[0], _segments_[1])
_segments_ = outerCircleTangents(circles[0], circles[2])
Q = rt.intersectionPoint(_segments_[0], _segments_[1])
_segments_ = outerCircleTangents(circles[2], circles[1])
P = rt.intersectionPoint(_segments_[0], _segments_[1])
# Step 4: Draw the tangent lines
_segments_ = circleTangents(circles[2], R)
S1,T1      = _segments_[0][1], _segments_[1][1]
_segments_ = circleTangents(circles[1], Q)
T2,S2      = _segments_[0][1], _segments_[1][1]
_segments_ = circleTangents(circles[0], P)
S3,T3      = _segments_[0][1], _segments_[1][1]
# Step 5: draw the inner and outer circles
_inner_ = rt.threePointCircle(S1,S2,S3)
_outer_ = rt.threePointCircle(T1,T2,T3)
svg_orthogonal_circle.append(f'<circle cx="{_inner_[0]}" cy="{_inner_[1]}" r="{_inner_[2]}" fill="none" stroke="black" stroke-width="0.1" />')
svg_orthogonal_circle.append(f'<circle cx="{_outer_[0]}" cy="{_outer_[1]}" r="{_outer_[2]}" fill="none" stroke="black" stroke-width="0.1" />')
svg_orthogonal_circle.append(svgX(S1,color='green')+svgX(S2,color='green')+svgX(S3,color='green'))
svg_orthogonal_circle.append(svgX(T1)+svgX(T2)+svgX(T3))

svg_circles = []
for i in range(len(circles)):
    svg_circles.append(f'<line x1="{_inner_[0]}" y1="{_inner_[1]}" x2="{circles[i][0]}" y2="{circles[i][1]}" stroke="#ff0000" stroke-width="0.2"/>')
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')

rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_orthogonal_circle) + svg_ftr,
         svg_hdr + ''.join(svg_example) + svg_ftr], spacer=10)

In [ ]:
#
# From ChatGPT (doesn't work -- see below)
#
# Write a python script to calculate the inscribed circle in the middle of three non-overlapping circles, c0, c1, and c2.  Each circle is described as a tuple (x, y, r).
#
import math

def calculate_inscribed_circle(c0, c1, c2):
    """
    Calculate the inscribed circle tangent to three given circles.

    Parameters:
        c0, c1, c2: Tuples representing the circles (x, y, r),
                    where x and y are the circle's center coordinates, and r is the radius.

    Returns:
        A tuple (x, y, r) representing the inscribed circle's center and radius.
    """
    def curvature(x, y, r):
        return 1 / r if r > 0 else -1 / abs(r)

    # Extract circle data
    x0, y0, r0 = c0
    x1, y1, r1 = c1
    x2, y2, r2 = c2

    # Compute curvatures (k0, k1, k2)
    k0 = curvature(x0, y0, r0)
    k1 = curvature(x1, y1, r1)
    k2 = curvature(x2, y2, r2)

    # Calculate curvature of the inscribed circle (k3) using Descartes Circle Theorem
    k3 = k0 + k1 + k2 + 2 * math.sqrt(k0 * k1 + k1 * k2 + k2 * k0)

    # Compute weighted center of the inscribed circle
    x3 = (k0 * x0 + k1 * x1 + k2 * x2 + 2 * math.sqrt(k0 * k1 + k1 * k2 + k2 * k0) * (x0 + x1 + x2)) / k3
    y3 = (k0 * y0 + k1 * y1 + k2 * y2 + 2 * math.sqrt(k0 * k1 + k1 * k2 + k2 * k0) * (y0 + y1 + y2)) / k3

    # Compute radius of the inscribed circle (r3)
    r3 = 1 / k3

    return (x3, y3, r3)

inscribed_circle = calculate_inscribed_circle(circles[0], circles[1], circles[2])
print(inscribed_circle)
svg_chatgpt = []
svg_chatgpt.append(f'<circle cx="{inscribed_circle[0]}" cy="{inscribed_circle[1]}" r="{inscribed_circle[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_chatgpt.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_chatgpt.append(rt.svgText('ChatGPT', 5, 123))

rt.tile([svg_hdr + ''.join(svg_chatgpt) + svg_ftr,
         svg_hdr + ''.join(svg_example) + svg_ftr], spacer=10)

In [ ]:
#
# Gemini Flash 2.0 Experimental -- a little better / every result seems to be within the three circles
#
import math

def distance(p1, p2):
    """Calculates the Euclidean distance between two points."""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def inscribed_circle(c0, c1, c2):
    """
    Calculates the inscribed circle tangent to three non-overlapping circles.

    Args:
        c0: Tuple (x, y, r) representing the first circle.
        c1: Tuple (x, y, r) representing the second circle.
        c2: Tuple (x, y, r) representing the third circle.

    Returns:
        A tuple (x, y, r) representing the inscribed circle, or None if it 
        cannot be calculated (e.g., overlapping input circles, degenerate cases).
        Returns None if the input circles are overlapping or otherwise do not form a proper triangle.
    """

    # Check for invalid input (overlapping or near-overlapping circles)
    if (distance((c0[0], c0[1]), (c1[0], c1[1])) <= c0[2] + c1[2] or
        distance((c0[0], c0[1]), (c2[0], c2[1])) <= c0[2] + c2[2] or
        distance((c1[0], c1[1]), (c2[0], c2[1])) <= c1[2] + c2[2]):
        return None

    # Calculate the radii of the 'inner' circles (adjusted for tangency)
    r0 = c0[2]
    r1 = c1[2]
    r2 = c2[2]

    # Calculate the distances between the centers
    a = distance((c1[0], c1[1]), (c2[0], c2[1]))
    b = distance((c0[0], c0[1]), (c2[0], c2[1]))
    c = distance((c0[0], c0[1]), (c1[0], c1[1]))

    # Heron's formula for the area of the triangle formed by the centers
    s = (a + b + c) / 2
    A = math.sqrt(s * (s - a) * (s - b) * (s - c))

    if A == 0: #Degenerate Triangle
        return None

    # Inradius of the triangle formed by the centers
    r = A / s

    # Calculate the coordinates of the incenter (center of the inscribed circle of the triangle formed by the centers)
    x = (a * c0[0] + b * c1[0] + c * c2[0]) / (a + b + c)
    y = (a * c0[1] + b * c1[1] + c * c2[1]) / (a + b + c)

    return (x, y, r)

inscribed = inscribed_circle(circles[0], circles[1], circles[2])

print(inscribed)
svg_gemini = []
svg_gemini.append(f'<circle cx="{inscribed[0]}" cy="{inscribed[1]}" r="{inscribed[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_gemini.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_gemini.append(rt.svgText('Gemini 2.0 Flash', 5, 123))

rt.tile([svg_hdr + ''.join(svg_gemini) + svg_ftr,
         svg_hdr + ''.join(svg_example) + svg_ftr], spacer=10)

In [ ]:
#
# Gemini 1.5 Flash // returns None... so, doesn't really work correctly...
#
import math

def inscribed_circle(c0, c1, c2):
  """
  Calculates the inscribed circle in the middle of three non-overlapping circles.

  Args:
    c0: Tuple representing the first circle (x, y, r).
    c1: Tuple representing the second circle (x, y, r).
    c2: Tuple representing the third circle (x, y, r).

  Returns:
    Tuple representing the inscribed circle (x, y, r), or None if no solution exists.
  """

  def distance(p1, p2):
    """Calculates the distance between two points."""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

  def intersection(c1, c2):
    """Calculates the intersection points of two circles."""
    x1, y1, r1 = c1
    x2, y2, r2 = c2
    d = distance((x1, y1), (x2, y2))

    if d > r1 + r2:
      return None  # Circles do not intersect

    a = (r1**2 - r2**2 + d**2) / (2 * d)
    h = math.sqrt(r1**2 - a**2)
    x3 = x1 + a * (x2 - x1) / d
    y3 = y1 + a * (y2 - y1) / d
    x4 = x3 + h * (y2 - y1) / d
    y4 = y3 - h * (x2 - x1) / d
    x5 = x3 - h * (y2 - y1) / d
    y5 = y3 + h * (x2 - x1) / d
    return [(x4, y4), (x5, y5)]

  # Find intersection points of the circles
  int12 = intersection(c0, c1)
  int13 = intersection(c0, c2)
  int23 = intersection(c1, c2)

  if not int12 or not int13 or not int23:
    return None  # No intersection points found

  # Calculate the center of the inscribed circle
  x_sum = 0
  y_sum = 0
  for p1, p2 in [(int12[0], int12[1]), (int13[0], int13[1]), (int23[0], int23[1])]:
    x_sum += (p1[0] + p2[0])
    y_sum += (p1[1] + p2[1])
  x_center = x_sum / 6
  y_center = y_sum / 6

  # Calculate the radius of the inscribed circle
  radius = distance((x_center, y_center), int12[0])

  return (x_center, y_center, radius)


inscribed = inscribed_circle(circles[0], circles[1], circles[2])

print(inscribed)
'''svg_gemini15 = []
svg_gemini15.append(f'<circle cx="{inscribed[0]}" cy="{inscribed[1]}" r="{inscribed[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_gemini15.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')

rt.tile([svg_hdr + ''.join(svg_gemini15) + svg_ftr,
         svg_hdr + ''.join(svg_example)  + svg_ftr], spacer=10)'''

In [ ]:
#
# Claude 3.5 Sonnet // Similar to the Gemini 2.0 version -- circle always in the middle but rarely (if ever) the right size
#
import math
from typing import Tuple

def calculate_inscribed_circle(c0: Tuple[float, float, float], 
                             c1: Tuple[float, float, float], 
                             c2: Tuple[float, float, float]) -> Tuple[float, float, float]:
    """
    Calculate the inscribed circle that touches three non-overlapping circles.
    
    Args:
        c0, c1, c2: Tuples of (x, y, r) representing circle center coordinates and radius
        
    Returns:
        Tuple of (x, y, r) for the inscribed circle
    """
    x0, y0, r0 = c0
    x1, y1, r1 = c1
    x2, y2, r2 = c2
    
    # Calculate the distances between circle centers
    d01 = math.sqrt((x1 - x0)**2 + (y1 - y0)**2)
    d02 = math.sqrt((x2 - x0)**2 + (y2 - y0)**2)
    d12 = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    # Check if circles overlap
    if d01 <= r0 + r1 or d02 <= r0 + r2 or d12 <= r1 + r2:
        raise ValueError("Circles must not overlap")
    
    # Calculate parameters for the quadratic equation
    a0 = 1.0 / r0
    a1 = 1.0 / r1
    a2 = 1.0 / r2
    
    # Calculate circle inversions
    k0 = x0**2 + y0**2 - r0**2
    k1 = x1**2 + y1**2 - r1**2
    k2 = x2**2 + y2**2 - r2**2
    
    # Calculate determinants
    det = (
        2 * (x0*y1 + x1*y2 + x2*y0 - x0*y2 - x1*y0 - x2*y1)
    )
    
    if abs(det) < 1e-10:
        raise ValueError("Circles are collinear or degenerate")
    
    # Calculate the center coordinates of the inscribed circle
    x = (k0*(y1 - y2) + k1*(y2 - y0) + k2*(y0 - y1)) / det
    y = (k0*(x2 - x1) + k1*(x0 - x2) + k2*(x1 - x0)) / det
    
    # Calculate the radius using one of the original circles
    r = abs(math.sqrt((x - x0)**2 + (y - y0)**2) - r0)
    
    return (x, y, r)

inscribed = calculate_inscribed_circle(circles[0], circles[1], circles[2])

print(inscribed)
svg_claude = []
svg_claude.append(f'<circle cx="{inscribed[0]}" cy="{inscribed[1]}" r="{inscribed[2]}" fill="none" stroke="black" stroke-width="1.0" />')
for i in range(len(circles)):
    svg_claude.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1" />')
svg_claude.append(rt.svgText('Claude', 5, 123))

rt.tile([svg_hdr + ''.join(svg_claude)  + svg_ftr,
         svg_hdr + ''.join(svg_gemini)  + svg_ftr,
         svg_hdr + ''.join(svg_chatgpt) + svg_ftr,
         svg_hdr + ''.join(svg_example) + svg_ftr], spacer=10)